In [1]:
import datasets
from utils import *
import numpy as np
import bert

/home/lynie/miniconda3/envs/eff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bert

In [2]:
bert_config = ModelArguments()
bert_config.num_labels = 4
bert_config.model = "bert-base-uncased"
bert_config.cache_size = 16
bert_config.batch_size = 8
bert_config.num_epochs = 5

## agnews

In [8]:
# split into train, val, test
set_seed(42)
data = datasets.load_from_disk("./data/10000_sampled_agnews")

llama_labels = open("./llama_results/10000_sampled_agnews_llama.txt", "r").readlines()
llama_labels = [int(l.strip()) for l in llama_labels]
total, correct = 0, 0
for i, d in enumerate(data):
    if data[i]['label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

def update_labels(example, idx):
    example['llm_label'] = llama_labels[idx]
    return example

data = data.map(update_labels, with_indices=True)
total, correct = 0, 0
for i, d in enumerate(data):
    if data[i]['llm_label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

# split data into train and test
original_data = data.train_test_split(test_size=0.5)

original_train_data = original_data['train'].shuffle()

results = []
for i in np.arange(0.1, 0.9, 0.05):
    bert_model = bert.BertModel(bert_config)

    train_data = original_train_data.train_test_split(train_size=i, shuffle=False)
    dataset = GenericDataset(train_data['train'])
    train_dataloader = DataLoader(dataset, batch_size=bert_config.batch_size, shuffle=True)

    val_dataset = GenericDataset(original_data['test'])
    val_dataloader = DataLoader(val_dataset, batch_size=bert_config.batch_size, shuffle=False)

    # train model
    acc = bert_model.train(train_dataloader, val_dataloader)
    results.append((i, acc))

LLAMA Accuracy: 0.6866
LLAMA Accuracy: 1.0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Model loaded
Epoch:  1


Epoch 0 Loss: 0.8983: 100%|██████████| 63/63 [00:16<00:00,  3.90it/s]


Epoch:  2


...

Epoch:  4


Epoch 3 Loss: 0.0206: 100%|██████████| 407/407 [01:49<00:00,  3.71it/s]


Epoch:  5


100%|██████████| 625/625 [01:02<00:00,  9.98it/s]


Validation Accuracy:  0.6928


In [10]:
results

[(0.1, 0.7508),
 (0.15000000000000002, 0.7272),
 (0.20000000000000004, 0.6932),
 (0.25000000000000006, 0.6992),
 (0.30000000000000004, 0.6906),
 (0.3500000000000001, 0.7136),
 (0.40000000000000013, 0.7284),
 (0.45000000000000007, 0.643),
 (0.5000000000000001, 0.7206),
 (0.5500000000000002, 0.6826),
 (0.6000000000000002, 0.7062),
 (0.6500000000000001, 0.6928)]

## openbook

In [3]:
import datasets
data = datasets.load_from_disk("./data/openbook")
def update_labels(example, idx):
    label_to_idx = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    example['label'] = label_to_idx[example['answerKey']]
    example['text'] = '''FACT: {} \n QUESTION: {}\n A: {}\n B: {}\n C: {}\n D: {}'''.format(example['fact1'], example['question_stem'], example['choices']['text'][0], example['choices']['text'][1], example['choices']['text'][2], example['choices']['text'][3])
    return example
data = data.map(update_labels, with_indices=True)

# split into train, val, test
set_seed(42)
train_data = data['train']
llama_labels = open("./llama_results/openbook_predictions.txt", "r").readlines()
llama_labels = [int(l.strip()) for l in llama_labels]
total, correct = 0, 0
for i, d in enumerate(train_data):
    if train_data[i]['label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

def update_labels(example, idx):
    example['llm_label'] = llama_labels[idx]
    return example

train_data = train_data.map(update_labels, with_indices=True)
total, correct = 0, 0
for i, d in enumerate(train_data):
    if train_data[i]['llm_label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

# split data into train and test
train_data = train_data.shuffle()
original_data = train_data.train_test_split(test_size=0.3)

# results = []
# for i in np.arange(0.1, 0.9, 0.05):
bert_model = bert.BertModel(bert_config)

dataset = GenericDataset(original_data['train'])
train_dataloader = DataLoader(dataset, batch_size=bert_config.batch_size, shuffle=True)

val_dataset = GenericDataset(original_data['test'])
val_dataloader = DataLoader(val_dataset, batch_size=bert_config.batch_size, shuffle=False)

# train model
acc = bert_model.train(train_dataloader, val_dataloader)
# results.append((i, acc))

LLAMA Accuracy: 0.6177123260036312
LLAMA Accuracy: 1.0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Model loaded
Epoch:  1


Epoch 0 Loss: 1.4538: 100%|██████████| 434/434 [01:55<00:00,  3.77it/s]


Epoch:  2


...

Epoch:  4


Epoch 3 Loss: 1.6048: 100%|██████████| 434/434 [01:57<00:00,  3.70it/s]


Epoch:  5


100%|██████████| 186/186 [00:18<00:00,  9.96it/s]

Validation Accuracy:  0.28360215053763443


In [ ]:
logits = bert_model.inference(val_dataloader)
preds = np.argmax(logits.cpu(), axis=1)

In [24]:
correct, total = 0, 0
for idx in range(len(preds)):
    if logits[idx].cpu().max().item() > 0.8:
        if preds[idx].item() == original_data['test']['label'][idx]:
            correct += 1
        total += 1
print(f"Accuracy: {correct/total}")

Accuracy: 0.29245283018867924
